# Yolo real-time accident detection

In [ ]:
#run this cell only
from collections import defaultdict
import math
import cv2
import numpy as np
from ultralytics import YOLO
from cap_from_youtube import cap_from_youtube 
import uuid
import datetime
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import storage
import torch


#this json needs to be in local files 
cred = credentials.Certificate("saferoads-2026-firebase-adminsdk-kz932-9806597ba7.json") 
app = firebase_admin.initialize_app(cred,{'storageBucket': 'saferoads-2026.appspot.com'})
firestore_client = firestore.client()
bucket = storage.bucket()

id = "machine_10" #This id needs to be given to the client apps(Android/iOS/Web) for tracking
print(id)

url=""
doc_ref = firestore_client.collection("machines").document(id)
doc = doc_ref.get()
if doc.exists:
    try:
        url = doc.to_dict()['camera'] # Gets the camera id/url from the user        
    except:
        url=""
else:
    print('No such document!')
doc_ref.set({"id": id,"status": "idle","start":False,"camera":url})
source_error=False
disconnected=False
while(True) :
    if not disconnected:
        doc = doc_ref.get()
        if doc.exists:
            start_cond = doc.to_dict()['start']# Gets the start/stop signal from the user
            url = doc.to_dict()['camera'] # Gets the camera id/url from the user              
        else:
            print('No such document!')

        if not start_cond:     
            doc_ref.update({"id": id,"status": "idle"})
            print("No start signal recieved")
        else:    
            frame_fetcher=0
            try:
                if 'youtube' in url:#if the url is from youtube then using cap_from_youtube package to import video
                    cap = cap_from_youtube(url, 'best')  
                else:
                    vurl=url+"/video"
                    cap = cv2.VideoCapture(vurl)
            except:
                doc_ref.update({"id": id,"status": "Camera_source_error"})    
                source_error=True  

            width  = cap.get(3)
            height = cap.get(4)
            print('width, height:', width, height)
            video_area=width*height
            if int(video_area) == 0:
                doc_ref.update({"id": id,"status": "Camera_source_error"})    
                source_error=True            

            if not source_error:      
                try:
                    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")                     
                    #model = YOLO('yolov8m.pt')#moderate accuracy but require medium hardware for realtime at 720p i.e RTX 4050                    
                    #model = YOLO('yolov8l.pt')#more accurate results but requires good hardware for realtime at 720p i.e RTX 4060                   
                    model = YOLO('yolov8x.pt')#most accurate results but requires good hardware for realtime at 720p i.e RTX 4070
                    model.to(device=device)
                except:
                    doc_ref.update({"id": id,"status": "Machine_error"})
                    break
                track_history = defaultdict(lambda: [])# Store the track history
                speed_history = defaultdict(lambda: [])# Store the relative history
                frame_skipper=0

                #when camera connected ,machine status is set to active
                doc_ref.update({"status": "active"})

                obj=[-1,-1]

                while cap.isOpened() :
                    # Break the loop if 'q' is pressed
                    if cv2.waitKey(30) & 0xFF == ord("q"):
                        doc_ref.update({"status": "Machine_disconnected"})
                        disconnected=True#if disconnected ,user must restart kernal 
                        cap.release()
                        cv2.destroyAllWindows() 
                        break

                    success, frame = cap.read()
                    if frame_fetcher <60:#fetches from the firebase after every 60 frame
                        frame_fetcher+=1
                    else: 
                        frame_fetcher =0
                        doc = doc_ref.get()
                        if doc.exists:
                            start_cond = doc.to_dict()['start']
                        else:
                            print('No such document!')

                    if success and start_cond:
                        #tracking only cars,trucks,buses,motorcycle and have a confidence more than 0.7
                        results = model.track(frame,conf=0.75, classes=[1,2,3,5,7],persist=True)
                        boxes = results[0].boxes.xywh

                        try:
                            class_ids=results[0].boxes.cls
                            track_ids = results[0].boxes.id.int().tolist()
                        except:
                            track_ids=None

                        annotated_frame = results[0].plot()

                        if track_ids is not None:
                            for box, track_id ,class_id in zip(boxes, track_ids,class_ids):
                                min_dist=1000
                                x, y, w, h = box        
                                area=float(w)* float(h)
                                track = track_history[track_id]
                                track.append((float(x), float(y)))  # x, y center point
                                if len(track) > 15:  # retain 7 tracks for 7 frames 
                                    track.pop(0)

                                for box1, track_id1 ,class_id1 in zip(boxes, track_ids,class_ids):
                                    if track_id != track_id1:#skipping its own track id
                                        x1, y1, w1, h1 = box1
                                        area1=float(w1)* float(h1)
                                        point=(float(x), float(y))
                                        point1=(float(x1), float(y1))
                                        dist=(math.dist(point,point1))/((math.sqrt(area)+math.sqrt(area1))/2)
                                         #relative distance between the points       
                                        if dist<min_dist:
                                            min_dist=dist
                                            obj[0]=int((class_id))
                                            obj[1]=int((class_id1))
                                # Draw the tracking lines
                                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                                distance_draw=math.dist(track[0],track[-1])
                                if distance_draw==0:
                                    relative_speed=0
                                else:    
                                    relative_speed=distance_draw/area

                                speed=speed_history[track_id]   
                                speed.append(relative_speed)
                                if len(speed) > 5:  # retain 5 tracks for 5 frames 
                                    speed.pop(0)

                                speed_difference=speed[-1]-speed[0]

                                if speed_difference==0:
                                    acceleration=0
                                else:    
                                    acceleration=speed_difference /len(speed)  

                                #print( results[0].names)
                                #cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=5)
                                #cv2.putText(annotated_frame,str(int(relative_speed*10000)),(int(x-(w/2)), int((h/2)+y)),cv2.FONT_HERSHEY_SIMPLEX ,1, (0, 225, 0),2,cv2.LINE_AA)
                                #cv2.putText(annotated_frame,str(int(abs(acceleration)*20000)),(int(x+(w/2)), int((h/2)+y)),cv2.FONT_HERSHEY_SIMPLEX ,1, (0, 0, 225),2,cv2.LINE_AA)
                                #cv2.putText(annotated_frame,str(int(min_dist*100)),(int(x), int((h/2)+y)),cv2.FONT_HERSHEY_SIMPLEX ,1, (225,0, 25),2,cv2.LINE_AA)
                                #changing the values may vary the detection of accident
                                if ((relative_speed*10000)>7) and ((min_dist*100)<120)and ((abs(acceleration)*20000)>3):#during accident the car should have some speed and it should have some accident                            
                                    if frame_skipper>0:
                                        frame_skipper=frame_skipper-1
                                        continue
                                    else: 
                                        frame_skipper=25#skips accident detection for 25 frames after an accident is detected

                                    #cv2.putText(annotated_frame,"Accident",(int(x-(w/4)), int(y)),cv2.FONT_HERSHEY_SIMPLEX ,1, (12,0,225),2,cv2.LINE_AA)
                                    accident_id = str(uuid.uuid1())#Gets a new unique id at every acciddent
                                    cropped_image = annotated_frame[int(y-(h*1.5)):int(y+(h*1.5)), int(x-(w*1.5)):int(x+(w*1.5))]
                                    cv2.imwrite("accident_{}.jpg".format(accident_id), annotated_frame)#saves the file in your loacl machine
                                    cv2.imwrite("accident_{}.jpg".format(accident_id), cropped_image)#rewrite the file after cropped,if cropping is successful
                                    try:
                                        blob = bucket.blob("accident_{}.jpg".format(accident_id))                                        
                                        outfile='accident_{}.jpg'.format(accident_id) #local file path

                                        with open(outfile, 'rb') as my_file:
                                            blob.upload_from_file(my_file) #here the file is read by python and uploaded
                                        blob.make_public() #the file is made public so anyone with the link can open
                                        print(blob.public_url) #public link url is printed here

                                        doc_ref.collection("accidents").document(accident_id).set({"id":accident_id,"image" : blob.public_url,"date" : str(datetime.datetime.now().date()),"time" : str(datetime.datetime.now().time())[0:8],"vehicles_involved":obj,"camera":url})                        
                                    except:
                                        None


                        # Display the annotated frame
                        cv2.imshow("YOLOv8 Tracking", annotated_frame)

                    else:
                        doc_ref.update({"status": "idle"})
                        break
                cap.release()
                cv2.destroyAllWindows() 



1234
No such document!
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
No start signal recieved
[youtube] Extracting URL: https://www.youtube.com/watch?v=CTQErltXZqM
[youtub


0: 384x640 (no detections), 20.4ms
Speed: 1.5ms preprocess, 20.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.4ms
Speed: 1.0ms preprocess, 21.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.9ms
Speed: 1.0ms preprocess, 21.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.0ms
Speed: 1.0ms preprocess, 21.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.5ms
Speed: 1.0ms preprocess, 21.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.7ms
Speed: 1.0ms preprocess, 24.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.0ms
Speed: 1.0ms preprocess, 21.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 23.6ms
Speed: 1.0ms preprocess, 23.6ms i


0: 384x640 (no detections), 26.2ms
Speed: 1.0ms preprocess, 26.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 26.4ms
Speed: 1.5ms preprocess, 26.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 26.5ms
Speed: 1.0ms preprocess, 26.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 26.3ms
Speed: 1.0ms preprocess, 26.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 26.5ms
Speed: 1.0ms preprocess, 26.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 26.7ms
Speed: 2.0ms preprocess, 26.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.9ms
Speed: 1.1ms preprocess, 25.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 27.3ms
Speed: 1.0ms preprocess, 27.3ms i


0: 384x640 (no detections), 28.1ms
Speed: 1.5ms preprocess, 28.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.7ms
Speed: 2.3ms preprocess, 25.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 26.5ms
Speed: 1.0ms preprocess, 26.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 65.5ms
Speed: 1.0ms preprocess, 65.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 66.1ms
Speed: 2.0ms preprocess, 66.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 62.7ms
Speed: 1.0ms preprocess, 62.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 48.2ms
Speed: 2.0ms preprocess, 48.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 47.1ms
Speed: 2.0ms preprocess, 47.1ms i

Speed: 2.0ms preprocess, 22.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 23.7ms
Speed: 1.5ms preprocess, 23.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 23.2ms
Speed: 1.5ms preprocess, 23.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 23.0ms
Speed: 2.0ms preprocess, 23.0ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 23.5ms
Speed: 1.0ms preprocess, 23.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 24.0ms
Speed: 1.0ms preprocess, 24.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 23.4ms
Speed: 2.0ms preprocess, 23.4ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 24.0ms
Speed: 1.0ms preprocess, 24.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 22.9ms
Speed: 2


0: 384x640 2 cars, 24.9ms
Speed: 1.2ms preprocess, 24.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 32.8ms
Speed: 1.0ms preprocess, 32.8ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 33.8ms
Speed: 1.1ms preprocess, 33.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 32.8ms
Speed: 1.5ms preprocess, 32.8ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 32.8ms
Speed: 1.0ms preprocess, 32.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 33.6ms
Speed: 1.0ms preprocess, 33.6ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 32.7ms
Speed: 1.0ms preprocess, 32.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 33.1ms
Speed: 1.0ms preprocess, 33.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x

Speed: 1.0ms preprocess, 42.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 38.9ms
Speed: 0.8ms preprocess, 38.9ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 35.5ms
Speed: 1.5ms preprocess, 35.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 35.1ms
Speed: 0.0ms preprocess, 35.1ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 39.2ms
Speed: 1.0ms preprocess, 39.2ms inference, 5.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 41.1ms
Speed: 1.0ms preprocess, 41.1ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 32.2ms
Speed: 2.0ms preprocess, 32.2ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 37.2ms
Speed: 1.0ms preprocess, 37.2ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 32.5ms
Speed: 1

# install packages for this project

In [ ]:
#if you want to run on CPU just install this(cpu version is very slow)
!pip install opencv-python
!pip install ultralytics
!pip install firebase-admin
!pip install cap-from-youtube
!pip install youtube_dl

In [ ]:
#If you want to run on gpu
#install cuda 12.1 from nvidia if not present

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
!pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 --index-url https://download.pytorch.org/whl/cu121

In [ ]:
import torch
import torchvision
import torchaudio
print(torch.__version__)
print(torchvision.__version__)
print(torchaudio.__version__)
#if it shows like this then ok
#2.1.1+cu121
#0.16.1+cu121
#2.1.1+cu121

